In [2]:
import pandas as pd 
import os
import random 
from sklearn.metrics import confusion_matrix


In [23]:
# read the data 
with open('train.csv','r') as train_txt:
    train_txt=train_txt.read()


In [26]:
with open('test.csv','r') as test_txt:
    test_txt=test_txt.read()

In [20]:
#
txt_doc = train_txt.split('\n')
print(txt_doc[1])


"QUERIES FROM PHARMACY","please to verify instructions for drugname patients wi from pharmacy target reason for call details please to verify instructions for drugname patients will be picking up Rx timephrase clarified Rx with pharmacy lisa gilligan rn"


In [30]:
test_doc = test_txt.split('\n')
print(test_doc[2])

"patients husband ret name spouse for other patients husband ret says you reach patients at timephrase am rna follow scheduled timephrase am"


In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics

In [76]:
df1=pd.read_csv('train.csv')
df1.shape
df2=pd.read_csv('test.csv')
df2.shape

(8581, 1)

In [96]:
df1.converse=df1.converse.astype(str)
df2=df2.astype(str)

In [98]:
df2.isnull().any()

converse    False
dtype: bool

In [82]:
tags=df1.converse
text=df1.categories

In [265]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df1.converse)
train_x = tokenizer.texts_to_sequences(df1.converse)
train_dc = pad_sequences(train_x, maxlen=50)
test_x = tokenizer.texts_to_sequences(df2.converse)
test_dc = pad_sequences(test_x, maxlen=50)
print(train_dc.shape,test_dc.shape)

(48699, 50) (8581, 50)


In [266]:
from keras.utils.np_utils import to_categorical
unique_labels=list(df1.categories.unique())
train_y=np.array([unique_labels.index(i) for i in df1.categories])
train_y = to_categorical(train_y)

In [267]:
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
#embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(10000,128))
#model.add(LSTM(128,input_shape=(50,1),return_sequences=True))
model.add(LSTM(128,input_shape=(50,1),return_sequences=False))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 128)         1280000   
_________________________________________________________________
lstm_16 (LSTM)               (None, None, 128)         131584    
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_11 (Dense)             (None, 21)                2709      
Total params: 1,545,877
Trainable params: 1,545,877
Non-trainable params: 0
_________________________________________________________________
None


In [268]:
#fitting the model 
model.fit(train_dc,train_y,epochs=5, batch_size=60)

Epoch 1/5
48699/48699 [==============================] - 426s - loss: 1.5414 - acc: 0.5256   
Epoch 2/5
48699/48699 [==============================] - 418s - loss: 1.2120 - acc: 0.6212   
Epoch 3/5
48699/48699 [==============================] - 421s - loss: 1.1052 - acc: 0.6516   
Epoch 4/5
48699/48699 [==============================] - 457s - loss: 1.0314 - acc: 0.6760   
Epoch 5/5
48699/48699 [==============================] - 422s - loss: 0.9810 - acc: 0.6890   


In [255]:
#prediction with the classes 
pre = model.predict_classes(test_dc)
len(pre)

8581/8581 [==============================] - 50s    


8581

In [325]:
pred=pd.DataFrame(pre)
#pred

In [211]:
un_labels=pd.DataFrame(unique_labels)
#print(un_labels)

In [270]:
#adding names to the lables 
lists=[]
for i in pre:
    lists.append(unique_labels[i])
    
    


In [246]:
print(lists)

['MEDICATION RELATED', 'NEW APPOINTMENT', 'MEDICATION RELATED', 'REFILL', 'NEW APPOINTMENT', 'MEDICATION RELATED', 'MEDICATION RELATED', 'NEW APPOINTMENT', 'OTHERS', 'MEDICATION RELATED', 'NEW APPOINTMENT', 'NEW APPOINTMENT', 'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)', 'LAB RESULTS', 'OTHERS', 'REFILL', 'REFILL', 'PROVIDER', 'LAB RESULTS', 'MEDICATION RELATED', 'MEDICATION RELATED', 'MEDICATION RELATED', 'MEDICATION RELATED', 'REFILL', 'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)', 'PROVIDER', 'REFILL', 'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)', 'OTHERS', 'MEDICATION RELATED', 'OTHERS', 'MEDICATION RELATED', 'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)', 'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)', 'OTHERS', 'MEDICATION RELATED', 'PRIOR AUTHORIZATION', 'NEW APPOINTMENT', 'REFILL', 'NEW APPOINTMENT', 'NEW APPOINTMENT', 'OTHERS', 'OTHERS', 'MEDICATION RELATED', 'REFILL', 'MEDICATION RELATED', 'NEW APPOINTMENT', 'PROVIDER', 'REFILL', 'OTHERS', 'QUERIES FROM PHARMACY', '

In [272]:
lists=pd.DataFrame(lists)

In [273]:
lists.to_csv('predictions.csv')

# cnn model 

In [312]:
def get_cnn_model_v3():    # added filter
    model1 = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model1.add(Embedding(10000,128))
    model1.add(Dropout(0.2))
    model1.add(Conv1D(128,3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model1.add(GlobalMaxPooling1D())
    model1.add(Dense(256))
    model1.add(Dropout(0.2))
    model1.add(Activation('relu'))
    model1.add(Dense(21))
    model1.add(Activation('softmax'))
    model1.summary()
    model1.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    return model1

In [313]:
m = get_cnn_model_v3()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, None, 128)         1280000   
_________________________________________________________________
dropout_17 (Dropout)         (None, None, 128)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 128)         49280     
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 128)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_15 (Activation)   (None, 256)               0         
__________

In [315]:
fit1=m.fit(train_dc,train_y,epochs=3, batch_size=100)


Epoch 1/3
48699/48699 [==============================] - 82s - loss: 1.0610 - acc: 0.6656    
Epoch 2/3
48699/48699 [==============================] - 84s - loss: 0.9663 - acc: 0.6940    
Epoch 3/3
48699/48699 [==============================] - 87s - loss: 0.8799 - acc: 0.7158    


In [321]:
cnn=m.predict_classes(test_dc)
print(cnn)

8544/8581 [============================>.] - ETA: 0s[3 1 3 ..., 3 4 4]


In [322]:
results=[]
for i in cnn:
    results.append(unique_labels[i])
    

In [324]:
pd.DataFrame(results).to_csv('predictions.csv')